# Uchazeč: data o uchazečích o studium na pedagogických fakultách

- přejmenuj sloupce v datasetu, aby to mělo logičtější strukturu
- připrav variable_labels a value_labels pro export do staty

## Importy

In [1]:
# nejake standardni importy
import os
import sys
import pyreadstat
import pandas as pd
import numpy as np
import re
from statsmodels.stats.weightstats import DescrStatsW
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
# aby grafy byly rozumně čitelné na obrazovce
plt.rcParams['figure.dpi'] = 90
plt.rcParams['axes.titlesize'] = 11
plt.rcParams['figure.figsize'] = 10, 5
#plt.ioff()
import dbf

In [2]:
data_root = '/mnt/d/projects/idea/data'
path17 = 'uchazec/0022MUCH17P'
path21 = 'uchazec/0022MUCH21P'

## Load data

In [3]:
def loader(year=21):
    
    # year can be 17 or 21
    data_root = '/mnt/d/projects/idea/data'
    path = f'uchazec/0022MUCH{year}P'
    
    df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250', low_memory=False)
    
    # conversion to numeric
    for c in ['RMAT', 'STAT', 'STATB', 'IZOS', 'VYPR', 'ZAPS'] + [f'APRO{i}' for i in range(1, 4)]:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        
    # strip white spaces
    for c in ['OBSS', 'RID', 'PROGRAM'] + [f'OBOR{i}' for i in range(1, 6)]:
        df[c] = df[c].str.strip()
    
    to_rename = {
        'RDAT': 'dat_sber',
        'RID': 'fak_id',
        'ROD_KOD': 'id',
        'STATB': 'bydliste_stat',
        'OBECB': 'bydliste_obec',
        'PSCB': 'bydliste_psc',
        'STAT': 'stat',
        'ODHL': 'odhl',
        'IZOS': 'ss_izo',
        'OBSS': 'ss_obor',
        'RMAT': 'rmat',
        'TYP_ST': 'typ_st',
        'FORMA_ST': 'forma_st',
        'PROGRAM': 'program',
        'OBOR1': 'obor1',
        'OBOR2': 'obor2',
        'OBOR3': 'obor3',
        'OBOR4': 'obor4',
        'OBOR5': 'obor5',
        'APRO1': 'apro1',
        'APRO2': 'apro2',
        'APRO3': 'apro3',
        'DAT_REG': 'dat_reg',
        'VYPR': 'vypr',
        'DAT_VYPR': 'dat_vypr',
        'ZAPS': 'zaps',
        'DAT_ZAPS': 'dat_zaps'
    }

    to_drop = ['CHYV']
    
    value_labels = {}
    variable_labels = {}
    
    df = df.rename(columns=to_rename).drop(columns=to_drop)
    
    # label STAT, STATB: register AAST
    aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C21752AAST.xml'
    aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')
    aast['IDX'] = aast.index
    aast['ISO_ALPHA3'] = aast['SPC'].str[2:5]
    df = pd.merge(df.rename(columns={'stat': 'KOD'}), aast[['KOD', 'IDX']].rename(columns={'IDX': 'stat'}), 
                  how='left').drop(columns=['KOD'])
    df['stat_iso'] = df['stat']
    df = pd.merge(df.rename(columns={'bydliste_stat': 'KOD'}), aast[['KOD', 'IDX']].rename(columns={'IDX': 'bydliste_stat'}), 
                  how='left').drop(columns=['KOD'])
    df['bydliste_stat_iso'] = df['bydliste_stat']
    
    aast_dict = aast[['IDX', 'ZKR']].set_index('IDX')['ZKR'].to_dict()
    aast_iso_dict = aast[['IDX', 'ISO_ALPHA3']].set_index('IDX')['ISO_ALPHA3'].to_dict()
    value_labels['stat'] = aast_dict
    value_labels['stat_iso'] = aast_iso_dict
    value_labels['bydliste_stat'] = aast_dict
    value_labels['bydliste_stat_iso'] = aast_iso_dict
    
    # registers for ODHL, TYP_ST, FORMA_ST, VYPR, ZAPS
    # http://stistko.uiv.cz/dsia/cisel.html -- might be needed to update urls
    registers = {
        'odhl': 'http://stistko.uiv.cz/katalog/textdata/C21820MCPP.xml',
        'typ_st': 'http://stistko.uiv.cz/katalog/textdata/C21845PASP.xml',
        'forma_st' : 'http://stistko.uiv.cz/katalog/textdata/C2196PAFS.xml',
        'vypr': 'http://stistko.uiv.cz/katalog/textdata/C21925MCPR.xml',
        'zaps': 'http://stistko.uiv.cz/katalog/textdata/C21939MCZR.xml',
        # 'OBSS': 'http://stistko.uiv.cz/katalog/textdata/C113922AKSO.xml'
    }
    
    for c, url in registers.items():
        rg = pd.read_xml(url, encoding='cp1250', xpath='./veta')
        rg['IDX'] = rg.index
        df = pd.merge(df.rename(columns={c: 'KOD'}), rg[['KOD', 'IDX']].rename(columns={'IDX': c}), 
                      how='left').drop(columns=['KOD'])        
        rg_dict = rg[['IDX', 'TXT']].set_index('IDX')['TXT'].to_dict()
        value_labels[c] = rg_dict
    
    # gender etc.
    value_labels['gender'] = {0: 'Dívka', 1: 'Chlapec', 2: 'Neznámé'}
    df['je_rc'] = df['id'].str[4:6] == 'QQ'
    df['rc1'] = np.where(df['je_rc'], pd.to_numeric(df['id'].str[:2], errors='coerce'), np.nan)
    df['rc2'] = np.where(df['je_rc'], pd.to_numeric(df['id'].str[2:4], errors='coerce'), np.nan)
    df['gender'] = np.where(df['je_rc'], np.where(df['rc2'] > 50, 0, 1), 2)
    df['nar_rok'] = np.where(df['je_rc'], np.where(df['rc1'] < year - 5, 2000 + df['rc1'], 1900 + df['rc1']), np.nan)
    df['nar_mesic'] = np.where(df['je_rc'], np.where(df['rc2'] > 50, df['rc2'] - 50, df['rc2']), np.nan)
    df = df.drop(columns=['je_rc', 'rc1', 'rc2'])
    df['vek'] = 2000 + year - df['nar_rok'] + (9 - df['nar_mesic']) / 12
    
    # label OBSS - I am not including full school info
    #   -> actually, this give me slightly better match - probably because I am including also specializations that are not valid anymore
    li = []
    for y in range(1999, 2023):
        pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
        pdf['year'] = y
        li.append(pdf)
    ss = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)
    ss = ss.sort_values('year', ascending=False).drop_duplicates('IZO').sort_index()
    ss['OBOR'] = ss['OBOR'].str.strip()

    sss = ss[['IZO', 'VUSC']].drop_duplicates().reset_index(drop=True)       
    # SS is complicated -> too lengthy for stata value labels...
    # df['ss'] = df['ss_izo']
    df = pd.merge(df, sss[['IZO', 'VUSC']].rename(columns={'IZO': 'ss_izo', 'VUSC': 'ss_nuts'}), how='left')
    # df['ss_nuts'] = df['ss_izo']
    # value_labels['ss'] = sss[['IZO', 'ZAR_PLN']].set_index('IZO')['ZAR_PLN'].to_dict()
    # value_labels['ss_nuts'] = sss[['IZO', 'VUSC']].set_index('IZO')['VUSC'].to_dict()
    
    odhl_ss = {v: k for k, v in value_labels['odhl'].items()}['Střední škola']
    value_labels['ss_typ'] = {9: 'Není SŠ', 0: 'SOŠ', 1: 'Gymnázium', 2: 'Jiné'}
    df['ss_typ'] = np.where(df['odhl'] != odhl_ss, 9, np.where(df['ss_obor'] == '', 2,
                                                               np.where(df['ss_obor'].str.startswith('794'), 1, 0)))
    df['ss_gym_delka'] = np.where(df['ss_typ'] == 1, pd.to_numeric(df['ss_obor'].str[5], errors='coerce'), np.nan)

    regpro = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv', encoding='cp1250')
    regpro['RID'] = regpro['RID'].str.strip()
    regpro = regpro[['RID', 'ZAR_PLN', 'VS_PLN', 'UZEMI']]
    regpro['IDX'] = regpro.index
    regpro['FAK_S_VS'] = regpro['VS_PLN'] + ', ' + regpro['ZAR_PLN']
    
    vs = regpro[['VS_PLN']].drop_duplicates().sort_values('VS_PLN').reset_index(drop=True)
    vs['VS_IDX'] = vs.index
    
    regpro = pd.merge(regpro, vs)
    df = pd.merge(df, regpro[['IDX', 'RID', 'VS_IDX', 'UZEMI']]
                  .rename(columns={'IDX': 'fak_nazev', 'VS_IDX': 'vs_nazev', 'RID': 'fak_id', 'UZEMI': 'fak_nuts'}),
                  how='left')
    
    # df['fak_plny'] = df['fak_nazev']
    value_labels['fak_nazev'] = regpro[['IDX', 'ZAR_PLN']].set_index('IDX')['ZAR_PLN'].str.strip().to_dict()
    # value_labels['fak_plny'] = regpro[['IDX', 'FAK_S_VS']].set_index('IDX')['FAK_S_VS'].to_dict()
    value_labels['vs_nazev'] = vs[['VS_IDX', 'VS_PLN']].set_index('VS_IDX')['VS_PLN'].str.strip().to_dict()    
    
    # this is sensible only for 2017 data...
#     akko_xml = 'http://stistko.uiv.cz/katalog/textdata/C11240AKKO.xml'
#     akko = pd.read_xml(akko_xml, encoding='cp1250', xpath='./veta')
#     for i in [1, 2, 5]:
#         df[f'obor1_{i}'] = df['obor1'].str[:i]
    
#     akko1 = akko[akko['KOD'].str.len() == 1].copy()
#     akko1['KOD'] = pd.to_numeric(akko1['KOD'])
#     value_labels['obor1_1'] = akko1[['KOD', 'TXT']].set_index('KOD')['TXT'].to_dict()
    
#     akko2 = akko[akko['KOD'].str.len() == 2].copy()
#     akko2['KOD'] = pd.to_numeric(akko2['KOD'])
#     value_labels['obor1_2'] = akko2[['KOD', 'TXT']].set_index('KOD')['TXT'].to_dict()
    
#     akko5 = akko[akko['KOD'].str.len() == 5].copy().reset_index(drop=True)
#     akko5['IDX'] = akko5.index

#     df = pd.merge(df.rename(columns={'obor1_5': 'KOD'}), akko5[['KOD', 'IDX']].rename(columns={'IDX': 'obor1_5'}), 
#                   how='left').drop(columns=['KOD'])
#     value_labels['obor1_5'] = akko5[['IDX', 'TXT']].set_index('IDX')['TXT'].to_dict()    
    
    # program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
    # program['IDX'] = program.index
    # df = pd.merge(df.rename(columns={'program': 'KOD'}), program[['KOD', 'IDX']].rename(columns={'IDX': 'program'}), 
    #               how='left').drop(columns=['KOD'])    
    # value_labels['program'] = program[['IDX', 'NAZEV']].set_index('IDX')['NAZEV'].to_dict()
    
#     akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
#     akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
#     akvo['IDX'] = akvo.index
#     akvo_dict = akvo[['IDX', 'TXT']].set_index('IDX')['TXT'].to_dict()
        
#     for i in range(1, 6):
#         df = pd.merge(df.rename(columns={f'obor{i}': 'KOD'}), akvo[['KOD', 'IDX']].rename(columns={'IDX': f'obor{i}'}), 
#                       how='left').drop(columns=['KOD'])        
#         value_labels[f'obor{i}'] = akvo_dict
    
    variable_labels = {
        'id': 'Identifikátor uchazeče (kódované rodné číslo)',
        'gender': 'Pohlaví uchazeče',
        'nar_rok': 'Rok narození',
        'nar_mesic': 'Měsíc narození',
        'vek': 'Přibližný věk uchazeče',
        'stat': 'Státní příslušnost uchazeče',
        'stat_iso': 'Státní příslušnost uchazeče (ISO)',
        'bydliste_stat': 'Stát trvalého pobytu',
        'bydliste_stat_iso': 'Stát trvalého pobytu (ISO)',
        'bydliste_obec': 'Obec trvalého pobytu',
        'bydliste_psc': 'PSČ trvalého pobytu',
        'fak_id': 'Identifikátor fakulty',
        'fak_nazev': 'Název fakulty',
        # 'fak_plny': 'Název fakulty včetně VŠ',
        'fak_nuts': 'Lokalita fakulty',
        'vs_nazev': 'Název vysoké školy',
        'odhl': 'Odkud se uchazeč hlásí',
        'ss_izo': 'Identifikátor střední školy',
        # 'ss': 'Střední škola',
        'ss_nuts': 'NUTS region střední školy',
        'ss_obor': 'Obor studia střední školy',
        'ss_typ': 'Typ střední školy',
        'ss_gym_delka': 'Délka studia gymnázia',
        'rmat': 'Rok maturity',
        'typ_st': 'Typ studia',
        'forma_st': 'Forma studia',
        'vypr': 'Výsledek přijímacího řízení',
        'zaps': 'Výsledek zápisu',
        'program': 'Studijní program',
        'obor1': 'Studijní obor',
        'obor2': 'Studijní obor',
        'obor3': 'Studijní obor',
        'obor4': 'Studijní obor',
        'obor5': 'Studijní obor',
        'apro1': 'Aprobace',
        'apro2': 'Aprobace',
        'apro3': 'Aprobace',
        # 'obor1_1': 'Skupina programů podle prvního oboru',
        # 'obor1_2': 'Podskupina programů podle prvního oboru',
        # 'obor1_5': 'Program podle prvního oboru',
        'dat_sber': 'Rozhodné datum sběru',
        'dat_reg': 'Datum podání (registrace) přihlášky',
        'dat_vypr': 'Datum rozhodnutí o výsledku přijímacího řízení',
        'dat_zaps': 'Datum zápisu',
    }

    df = df[variable_labels.keys()]
    
    return df, variable_labels, value_labels
    


In [4]:
for year in [17, 21]:
    print(f'loading 20{year}')
    df, variable_labels, value_labels = loader(year=year)
    df.to_stata(f'{data_root}/uchazec/uch{year}.dta', write_index=False, version=118, variable_labels=variable_labels, value_labels=value_labels)

loading 2017
loading 2021


In [7]:
df, variable_labels, value_labels = loader()

In [5]:
df.shape

(284201, 39)

In [6]:
variable_labels

{'id': 'Identifikátor uchazeče (kódované rodné číslo)',
 'gender': 'Pohlaví uchazeče',
 'nar_rok': 'Rok narození',
 'nar_mesic': 'Měsíc narození',
 'vek': 'Přibližný věk uchazeče',
 'stat': 'Státní příslušnost uchazeče',
 'stat_iso': 'Státní příslušnost uchazeče (ISO)',
 'bydliste_stat': 'Stát trvalého pobytu',
 'bydliste_stat_iso': 'Stát trvalého pobytu (ISO)',
 'bydliste_obec': 'Obec trvalého pobytu',
 'bydliste_psc': 'PSČ trvalého pobytu',
 'fak_id': 'Identifikátor fakulty',
 'fak_nazev': 'Název fakulty',
 'fak_nuts': 'Lokalita fakulty',
 'vs_nazev': 'Název vysoké školy',
 'odhl': 'Odkud se uchazeč hlásí',
 'ss_izo': 'Identifikátor střední školy',
 'ss_nuts': 'NUTS region střední školy',
 'ss_obor': 'Obor studia střední školy',
 'ss_typ': 'Typ střední školy',
 'ss_gym_delka': 'Délka studia gymnázia',
 'rmat': 'Rok maturity',
 'typ_st': 'Typ studia',
 'forma_st': 'Forma studia',
 'vypr': 'Výsledek přijímacího řízení',
 'zaps': 'Výsledek zápisu',
 'program': 'Studijní program',
 'obor

In [7]:
for c in df.columns:
    if c in value_labels.keys():
        df[c] = df[c].map(value_labels[c]).astype('category')

In [8]:
df.to_parquet('temp/uchazec/uch21.parquet')

In [9]:
df.head()

,id,gender,nar_rok,nar_mesic,vek,stat,stat_iso,bydliste_stat,bydliste_stat_iso,bydliste_obec,...,obor3,obor4,obor5,apro1,apro2,apro3,dat_sber,dat_reg,dat_vypr,dat_zaps
0,,Neznámé,NaN,NaN,NaN,Slovensko,SVK,Slovensko,SVK,,...,,,,NaN,NaN,NaN,20211031,8072021,08072021,08072021
1,,Neznámé,NaN,NaN,NaN,Slovensko,SVK,Slovensko,SVK,,...,,,,NaN,NaN,NaN,20211031,23062021,23062021,23062021
2,,Neznámé,NaN,NaN,NaN,Slovensko,SVK,Slovensko,SVK,,...,,,,NaN,NaN,NaN,20211031,28072021,28072021,28072021
3,,Neznámé,NaN,NaN,NaN,Slovensko,SVK,Slovensko,SVK,,...,,,,NaN,NaN,NaN,20211031,8072021,08072021,08072021
4,,Neznámé,NaN,NaN,NaN,Nigérie,NGA,Nigérie,NGA,,...,,,,NaN,NaN,NaN,20211031,10112020,10112020,


In [ ]:
for year in [17, 21]:
    df, variable_labels, value_labels = loader(year=year)
    for c in df.columns:
        if c in value_labels.keys():
            df[c] = df[c].map(value_labels[c]).astype('category')
    df.to_parquet(f'temp/uchazec/uch{year}.parquet')

df17 = pd.read_parquet('temp/uchazec/uch17.parquet')
df21 = pd.read_parquet('temp/uchazec/uch21.parquet')
df = df21

## Read data

In [15]:
variable_labels = {'id': 'Identifikátor uchazeče (kódované rodné číslo)',
 'gender': 'Pohlaví uchazeče',
 'nar_rok': 'Rok narození',
 'nar_mesic': 'Měsíc narození',
 'vek': 'Přibližný věk uchazeče',
 'stat': 'Státní příslušnost uchazeče',
 'stat_iso': 'Státní příslušnost uchazeče (ISO)',
 'bydliste_stat': 'Stát trvalého pobytu',
 'bydliste_stat_iso': 'Stát trvalého pobytu (ISO)',
 'bydliste_obec': 'Obec trvalého pobytu',
 'bydliste_psc': 'PSČ trvalého pobytu',
 'fak_id': 'Identifikátor fakulty',
 'fak_nazev': 'Název fakulty',
 'fak_nuts': 'Lokalita fakulty',
 'vs_nazev': 'Název vysoké školy',
 'odhl': 'Odkud se uchazeč hlásí',
 'ss_izo': 'Identifikátor střední školy',
 'ss_nuts': 'NUTS region střední školy',
 'ss_obor': 'Obor studia střední školy',
 'ss_typ': 'Typ střední školy',
 'ss_gym_delka': 'Délka studia gymnázia',
 'rmat': 'Rok maturity',
 'typ_st': 'Typ studia',
 'forma_st': 'Forma studia',
 'vypr': 'Výsledek přijímacího řízení',
 'zaps': 'Výsledek zápisu',
 'program': 'Studijní program',
 'obor1': 'Studijní obor',
 'obor2': 'Studijní obor',
 'obor3': 'Studijní obor',
 'obor4': 'Studijní obor',
 'obor5': 'Studijní obor',
 'apro1': 'Aprobace',
 'apro2': 'Aprobace',
 'apro3': 'Aprobace',
 'dat_sber': 'Rozhodné datum sběru',
 'dat_reg': 'Datum podání (registrace) přihlášky',
 'dat_vypr': 'Datum rozhodnutí o výsledku přijímacího řízení',
 'dat_zaps': 'Datum zápisu'}

In [106]:
df = pd.read_parquet('temp/uchazec/uch21.parquet')

## Filtrování dat

In [18]:
df.shape

(284201, 39)

- QQ v id, pak zkontroluj kolik je opakovaných záznamů; tím bych měl mít také jenom českou národnost
- Hlásí se ze střední školy

Kolik žáků tím vyřadím?

- podíl úspěšných uchazečů podle krajů, typu školy
- k tomu ale potřebuji rozlišit "nepřijat kvůli zkoušce" a "přijímací řízení zrušeno"
    - možná jít až na úroveň jednotlivých kategorií vypr?

In [15]:
# QQ v id
(df['id'].str[4:6] == 'QQ').value_counts()

True     240372
False     43829
Name: id, dtype: int64

In [130]:
df = df[df['id'].str[4:6] == 'QQ']
df = df[df['stat_iso'] == 'CZE']
# df = df[df['ss_typ'].isin(['Gymnázium', 'SOŠ'])]
# jen žáci co letos maturují
df = df[df['rmat'] == 2021]
df = df.dropna(subset=['vypr'])
unused_cat = ['gender', 'stat_iso', 'ss_typ']
for uc in unused_cat:
    df[uc] = df[uc].cat.remove_unused_categories()
df = df.reset_index(drop=True)

In [108]:
url = 'http://stistko.uiv.cz/katalog/textdata/C213145AKEN.xml'
rg = pd.read_xml(url, encoding='cp1250', xpath='./veta')
nuts_dict = rg.set_index('KOD')['ZKR'].to_dict()
df['ss_kraj'] = df['ss_nuts'].str[:5].map(nuts_dict).astype('category')

In [124]:
df['vypr']

0               Nepřijat pro neprospěch u přijímací zkoušky
1                              Přijat bez přijímací zkoušky
2         Nepřijat pro nedostačující kapacitu oboru/stud...
3         Nepřijat pro nedostačující kapacitu oboru/stud...
4         Nepřijat pro nedostačující kapacitu oboru/stud...
                                ...                        
144556          Nepřijat pro neprospěch u přijímací zkoušky
144557                   Nedostavil se k přijímacímu řízení
144558                   Nedostavil se k přijímacímu řízení
144559          Nepřijat pro neprospěch u přijímací zkoušky
144560          Nepřijat pro neprospěch u přijímací zkoušky
Name: vypr, Length: 144561, dtype: category
Categories (12, object): ['Nedostavil se k přijímacímu řízení', 'Nepřijat pro nedoložení požadovaného předchoz..., 'Nepřijat pro nedostačující kapacitu oboru/stu..., 'Nepřijat pro neprospěch u přijímací zkoušky', ..., 'Přijat z kurzů celoživotního vzdělávání', 'Přijat z předchozího vysokoškolské

In [131]:
df['vypr'].str.startswith('Přijat').value_counts(dropna=False)

False    78633
True     65485
Name: vypr, dtype: int64

In [132]:
df['vypr'].value_counts(dropna=False)

Přijat na základě přijímací zkoušky                              32206
Přijat bez přijímací zkoušky                                     31507
Nepřijat pro nedostačující kapacitu oboru/studijního programu    24496
Nepřijat pro neprospěch u přijímací zkoušky                      23961
Nedostavil se k přijímacímu řízení                               14976
Nepřijat pro nedoložení požadovaného předchozího vzdělání        11523
Nepřijat pro nesplnění dalších podmínek přijetí                   2116
Přijat dodatečně (po odvolání, po přezkumném řízení, apod.)       1734
Přijímací řízení zrušeno z administrativních důvodů               1553
Přijat z předchozího vysokoškolského studia (přestup)               38
Přijímací řízení zrušeno pro nedostatek uchazečů                     8
Přijat z kurzů celoživotního vzdělávání                              0
Name: vypr, dtype: int64

In [127]:
df['vypr'].shape

(144561,)

In [133]:
df['vypr_flag'] = df['vypr'].str.startswith('Přijat')
df['zaps_neprijat'] = df['zaps'] == 'Uchazeč nebyl přijat'
df['zaps_zapsal'] = df['zaps'] == 'Uchazeč se zapsal'
df['zaps_nezapsal'] = df['zaps'] == 'Uchazeč se nezapsal'

In [147]:
other_keys = ['gender', 'ss_kraj', 'ss_typ', 'ss_gym_delka']
df_ids = df.groupby('id')[other_keys].first().reset_index()

In [111]:
df_ids.head()

,id,gender,ss_kraj,ss_typ,ss_gym_delka
0,0001QQ0455,Chlapec,Liberecký kraj,SOŠ,NaN
1,0001QQ0461,Chlapec,Hlavní město Praha,Není SŠ,NaN
2,0001QQ3490,Chlapec,Hlavní město Praha,SOŠ,NaN
3,0001QQ3964,Chlapec,Středočeský kraj,SOŠ,NaN
4,0001QQ6105,Chlapec,Plzeňský kraj,SOŠ,NaN


In [113]:
pedf_list = ['Pedagogická fakulta', 'Fakulta pedagogická']
df['pedf'] = df['fak_nazev'].isin(pedf_list)
df['ones'] = 1

In [144]:
variables = ['ones', 'vypr_flag', 'zaps_zapsal']
cols = ['prihl_nepedf', 'prihl_pedf', 'prijat_nepedf', 'prijat_pedf', 'zapis_nepedf', 'zapis_pedf']
foo = df.groupby(['id', 'pedf'])[variables].sum().unstack('pedf')
foo.columns = cols
foo = foo.fillna(0).reset_index()

In [150]:
ff = pd.merge(df_ids, foo)

In [152]:
ff.to_parquet('temp/uchazec/ff21.parquet')

In [154]:
ff.head()

,id,gender,ss_kraj,ss_typ,ss_gym_delka,prihl_nepedf,prihl_pedf,prijat_nepedf,prijat_pedf,zapis_nepedf,zapis_pedf
0,0001QQ0455,Chlapec,Liberecký kraj,SOŠ,NaN,1.0,0.0,1.0,0.0,1.0,0.0
1,0001QQ0461,Chlapec,Hlavní město Praha,Není SŠ,NaN,2.0,0.0,2.0,0.0,2.0,0.0
2,0001QQ3490,Chlapec,Hlavní město Praha,SOŠ,NaN,1.0,1.0,0.0,0.0,0.0,0.0
3,0001QQ3964,Chlapec,Středočeský kraj,SOŠ,NaN,1.0,0.0,0.0,0.0,0.0,0.0
4,0001QQ6105,Chlapec,Plzeňský kraj,SOŠ,NaN,1.0,0.0,1.0,0.0,1.0,0.0


In [161]:
ff[['prihl_nepedf', 'prihl_pedf']].value_counts().rename('count').reset_index().to_parquet('temp/uchazec/foo.parquet')

In [165]:
(ff[cols] > 0).value_counts().rename('count').reset_index().to_parquet('temp/uchazec/bools.parquet')

In [119]:
foo.columns = range(8)

In [145]:
foo

,id,prihl_nepedf,prihl_pedf,prijat_nepedf,prijat_pedf,zapis_nepedf,zapis_pedf
0,0001QQ0455,1.0,0.0,1.0,0.0,1.0,0.0
1,0001QQ0461,2.0,0.0,2.0,0.0,2.0,0.0
2,0001QQ3490,1.0,1.0,0.0,0.0,0.0,0.0
3,0001QQ3964,1.0,0.0,0.0,0.0,0.0,0.0
4,0001QQ6105,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
50470,9962QQ9751,1.0,1.0,0.0,0.0,0.0,0.0
50471,9962QQ9759,3.0,0.0,2.0,0.0,1.0,0.0
50472,9962QQ9780,1.0,3.0,1.0,1.0,1.0,1.0
50473,9962QQ9790,1.0,0.0,1.0,0.0,1.0,0.0


In [94]:
df.shape

(141055, 45)

In [57]:
df['ss_kraj'] = df['ss_nuts'].str[:5].map(nuts_dict).astype('category')

In [58]:
df['ss_kraj'].head(10)

0      Středočeský kraj
1       Pardubický kraj
2       Pardubický kraj
3       Pardubický kraj
4       Pardubický kraj
5       Pardubický kraj
6       Pardubický kraj
7     Jihomoravský kraj
8    Hlavní město Praha
9        Liberecký kraj
Name: ss_kraj, dtype: category
Categories (14, object): ['Hlavní město Praha', 'Jihomoravský kraj', 'Jihočeský kraj', 'Karlovarský kraj', ..., 'Plzeňský kraj', 'Středočeský kraj', 'Zlínský kraj', 'Ústecký kraj']

In [60]:
len(df['id'].unique())

50519

In [63]:
df['typ_st'].value_counts()

Bakalářský     123452
Magisterský     21109
Name: typ_st, dtype: int64

In [64]:
# ok, seems to be ok to restrict on Prezenční
df['forma_st'].value_counts()

Prezenční      138310
Kombinované      6176
Distanční          75
Name: forma_st, dtype: int64

In [65]:
df['vypr'].value_counts()

Přijat na základě přijímací zkoušky                              32206
Přijat bez přijímací zkoušky                                     31507
Nepřijat pro nedostačující kapacitu oboru/studijního programu    24496
Nepřijat pro neprospěch u přijímací zkoušky                      23961
Nedostavil se k přijímacímu řízení                               14976
Nepřijat pro nedoložení požadovaného předchozího vzdělání        11523
Nepřijat pro nesplnění dalších podmínek přijetí                   2116
Přijat dodatečně (po odvolání, po přezkumném řízení, apod.)       1734
Přijímací řízení zrušeno z administrativních důvodů               1553
Přijat z předchozího vysokoškolského studia (přestup)               38
Přijímací řízení zrušeno pro nedostatek uchazečů                     8
Přijat z kurzů celoživotního vzdělávání                              0
Name: vypr, dtype: int64

In [67]:
df['vypr_flag'] = df['vypr'].str.startswith('Přijat')

In [68]:
df['zaps'].value_counts()

Uchazeč nebyl přijat    79075
Uchazeč se zapsal       44115
Uchazeč se nezapsal     21370
Name: zaps, dtype: int64

In [69]:
df['zaps_neprijat'] = df['zaps'] == 'Uchazeč nebyl přijat'
df['zaps_zapsal'] = df['zaps'] == 'Uchazeč se zapsal'
df['zaps_nezapsal'] = df['zaps'] == 'Uchazeč se nezapsal'

In [70]:
df[['zaps_neprijat', 'zaps_zapsal', 'zaps_nezapsal']].sum(axis=0)

zaps_neprijat    79075
zaps_zapsal      44115
zaps_nezapsal    21370
dtype: int64

In [102]:
other_keys = ['gender', 'ss_kraj', 'ss_typ', 'ss_gym_delka']

In [103]:
df.groupby('id')[other_keys].first()

,gender,ss_kraj,ss_typ,ss_gym_delka
id,,,,
0001QQ0455,Chlapec,Liberecký kraj,SOŠ,NaN
0001QQ3490,Chlapec,Hlavní město Praha,SOŠ,NaN
0001QQ3964,Chlapec,Středočeský kraj,SOŠ,NaN
0001QQ6105,Chlapec,Plzeňský kraj,SOŠ,NaN
0001QQ6723,Chlapec,Moravskoslezský kraj,SOŠ,NaN
...,...,...,...,...
9962QQ9751,Dívka,Ústecký kraj,SOŠ,NaN
9962QQ9759,Dívka,Liberecký kraj,SOŠ,NaN
9962QQ9780,Dívka,Moravskoslezský kraj,SOŠ,NaN


In [95]:
keys = ['id', 'gender', 'ss_kraj', 'ss_typ', 'ss_gym_delka']

In [96]:
df[['id']].drop_duplicates().shape

(48931, 1)

In [97]:
df[keys].drop_duplicates().shape

(49029, 5)

In [98]:
foo = df[keys].drop_duplicates()['id'].value_counts()

In [99]:
dup_ids = foo[foo > 1].index

In [100]:
foo = df[keys].drop_duplicates()

In [101]:
foo[foo['id'].isin(dup_ids)].sort_values('id')

,id,gender,ss_kraj,ss_typ,ss_gym_delka
511,0006QQ6740,Chlapec,Královéhradecký kraj,Gymnázium,4.0
513,0006QQ6740,Chlapec,Královéhradecký kraj,Gymnázium,8.0
993,0009QQ3968,Chlapec,Kraj Vysočina,SOŠ,NaN
994,0009QQ3968,Chlapec,NaN,SOŠ,NaN
1329,0010QQ9674,Chlapec,Hlavní město Praha,SOŠ,NaN
...,...,...,...,...,...
140585,9954QQ9682,Dívka,Jihočeský kraj,SOŠ,NaN
140601,9955QQ0461,Dívka,Hlavní město Praha,Gymnázium,4.0
140602,9955QQ0461,Dívka,Hlavní město Praha,Gymnázium,8.0
140893,9961QQ7203,Dívka,NaN,SOŠ,NaN


In [89]:
df['ss_typ'].value_counts()

Gymnázium    74685
SOŠ          66370
Není SŠ       2914
Jiné           592
Name: ss_typ, dtype: int64

In [45]:
df['ss_nuts'][df['ss_nuts'].str.len() != 6]

106       None
189       None
266       None
329       None
336       None
          ... 
144192    None
144193    None
144242    None
144390    None
144478    None
Name: ss_nuts, Length: 861, dtype: object

In [47]:
df['ss_typ'][df['ss_nuts'].str.len() != 6]

106          Jiné
189           SOŠ
266          Jiné
329       Není SŠ
336           SOŠ
           ...   
144192       Jiné
144193       Jiné
144242        SOŠ
144390        SOŠ
144478        SOŠ
Name: ss_typ, Length: 861, dtype: category
Categories (4, object): ['Gymnázium', 'Jiné', 'Není SŠ', 'SOŠ']

In [38]:
df['stat_iso']

0         CZE
1         CZE
2         CZE
3         CZE
4         CZE
         ... 
144556    CZE
144557    CZE
144558    CZE
144559    CZE
144560    CZE
Name: stat_iso, Length: 144561, dtype: category
Categories (1, object): ['CZE']

In [28]:
df['fak_nazev']

0                 Celoškolská pracoviště
1         Fakulta chemicko-technologická
2             Fakulta ekonomicko-správní
3                    Fakulta filozofická
4                    Fakulta filozofická
                       ...              
144556               Filozofická fakulta
144557               Filozofická fakulta
144558               Pedagogická fakulta
144559               Filozofická fakulta
144560               Filozofická fakulta
Name: fak_nazev, Length: 144561, dtype: category
Categories (104, object): ['1. lékařská fakulta', '2. lékařská fakulta', '3. lékařská fakulta', 'Agronomická fakulta', ..., 'Teologická fakulta', 'Zahradnická fakulta', 'Zdravotně sociální fakulta', 'Zemědělská fakulta']

In [29]:
pedf_list = ['Pedagogická fakulta', 'Fakulta pedagogická']

In [30]:
df['pedf'] = df['fak_nazev'].isin(pedf_list)

In [31]:
df['pedf'].value_counts()

False    126800
True      17761
Name: pedf, dtype: int64

In [39]:
df['gender'].value_counts()

Dívka      90625
Chlapec    53936
Name: gender, dtype: int64

In [60]:
value_labels['gender']

{0: 'Dívka', 1: 'Chlapec', 2: 'Neznámé'}

In [61]:
df.to_parquet('temp/uchazec/uch21.parquet')

In [35]:
# celkové zastoupení dívek a chlapců
foo = df[['id', 'gender']].drop_duplicates()['gender']
foo.value_counts() / len(foo)

Dívka      0.571092
Chlapec    0.428908
Neznámé    0.000000
Name: gender, dtype: float64

In [72]:
# ale na pedf je poměr výrazně odlišný, dívek se hlásí násobně více
foo = df[df['pedf']][['id', 'gender']].drop_duplicates()['gender']
foo.map(value_labels['gender']).value_counts() / len(foo)

Dívka      0.771531
Chlapec    0.228469
Name: gender, dtype: float64

In [75]:
# celkové zastoupení dívek a chlapců
foo = df[['id', 'ss_typ']].drop_duplicates()['ss_typ']
foo.map(value_labels['ss_typ']).value_counts() / len(foo)

SOŠ          0.550914
Gymnázium    0.405088
Není SŠ      0.038137
Jiné         0.005861
Name: ss_typ, dtype: float64

In [76]:
# celkové zastoupení dívek a chlapců
foo = df[df['pedf']][['id', 'ss_typ']].drop_duplicates()['ss_typ']
foo.map(value_labels['ss_typ']).value_counts() / len(foo)

SOŠ          0.503004
Gymnázium    0.473068
Není SŠ      0.023012
Jiné         0.000916
Name: ss_typ, dtype: float64

In [ ]:
# zvláštní -- je tam podíl těch, kteří letos maturovali

In [77]:
value_labels['vypr']

{0: 'Přijat na základě přijímací zkoušky',
 1: 'Přijat bez přijímací zkoušky',
 2: 'Přijat dodatečně (po odvolání, po přezkumném řízení, apod.)',
 3: 'Přijat z předchozího vysokoškolského studia (přestup)',
 4: 'Přijat z kurzů celoživotního vzdělávání',
 5: 'Nepřijat pro neprospěch u přijímací zkoušky',
 6: 'Nepřijat pro nedostačující kapacitu oboru/studijního programu',
 7: 'Nepřijat pro nedoložení požadovaného předchozího vzdělání',
 8: 'Nepřijat pro nesplnění dalších podmínek přijetí',
 9: 'Přijímací řízení zrušeno pro nedostatek uchazečů',
 10: 'Přijímací řízení zrušeno z administrativních důvodů',
 11: 'Nedostavil se k přijímacímu řízení'}

In [86]:
value_labels['gender']

{0: 'Dívka', 1: 'Chlapec', 2: 'Neznámé'}

In [78]:
value_labels['zaps']

{0: 'Uchazeč nebyl přijat', 1: 'Uchazeč se zapsal', 2: 'Uchazeč se nezapsal'}

In [79]:
df['prj'] = df['vypr'] < 5
df['zps'] = df['zaps'] == 1

In [82]:
foo = df.groupby(['id', 'gender', 'pedf'])[['prj', 'zps']].any().reset_index()

In [87]:
foo.groupby(['gender', 'pedf'])['prj'].mean()

gender  pedf 
0       False    0.809422
        True     0.415686
1       False    0.893261
        True     0.394372
Name: prj, dtype: float64

In [89]:
foo.groupby(['gender', 'pedf', 'prj'])['zps'].mean()

gender  pedf   prj  
0       False  False    0.000000
               True     0.943307
        True   False    0.000000
               True     0.833598
1       False  False    0.000000
               True     0.965128
        True   False    0.000000
               True     0.783692
Name: zps, dtype: float64

In [88]:
foo.shape

(58064, 5)

In [48]:
df['pedf']

30        False
46        False
47        False
48        False
49        False
          ...  
284185    False
284186    False
284187    False
284190    False
284191    False
Name: pedf, Length: 144561, dtype: bool

In [49]:
df['fak_nazev'].map(value_labels['fak_nazev'])

30        Celoškolská pracoviště                        ...
46        Fakulta chemicko-technologická                ...
47        Fakulta ekonomicko-správní                    ...
48        Fakulta filozofická                           ...
49        Fakulta filozofická                           ...
                                ...                        
284185    Filozofická fakulta                           ...
284186    Filozofická fakulta                           ...
284187    Pedagogická fakulta                           ...
284190    Filozofická fakulta                           ...
284191    Filozofická fakulta                           ...
Name: fak_nazev, Length: 144561, dtype: object

In [50]:
df['fak_nazev'].map(value_labels['fak_nazev'])[284187]

'Pedagogická fakulta                                                             '

In [36]:
df.shape

(144561, 39)

In [25]:
df['id'].value_counts()

0158QQ2454    24
0159QQ9608    19
0107QQ6193    18
0208QQ8498    17
0206QQ8532    16
              ..
0157QQ9271     1
0157QQ9275     1
0157QQ9284     1
0157QQ9288     1
0001QQ3964     1
Name: id, Length: 50519, dtype: int64

In [28]:
df['id'].value_counts().value_counts()

2     13509
1     11984
3     10463
4      6760
5      3884
6      1929
7       985
8       464
9       253
10      133
11       61
12       42
13       21
14       12
15       12
16        3
19        1
17        1
18        1
24        1
Name: id, dtype: int64

In [33]:
df['fak_nazev'].map(value_labels['fak_nazev']).value_counts().reset_index().to_parquet(f'temp/uchazec/fak_nazev.parquet')

In [ ]:
df['fak_nazev'].map(value_labels['fak_nazev']).value_counts().reset_index().to_parquet(f'temp/uchazec/fak_nazev.parquet')

In [34]:
df['vs_nazev'].map(value_labels['vs_nazev']).value_counts().reset_index().to_parquet(f'temp/uchazec/vs_nazev.parquet')

In [35]:
vs_list = ['Univerzita Karlova', 'Masarykova univerzita']

In [20]:
df.shape

(231074, 39)

In [22]:
df['id'].value_counts()

7808QQ3962    44
0056QQ9720    26
0158QQ2454    24
0051QQ9591    19
0159QQ9608    19
              ..
7154QQ3968     1
7154QQ7212     1
7154QQ7211     1
7154QQ7213     1
0001QQ3964     1
Name: id, Length: 102733, dtype: int64

In [23]:
df['rmat'].value_counts()

2021.0    144561
2020.0     33613
2019.0     12375
2018.0      7931
2017.0      5289
2016.0      4420
2015.0      2845
2013.0      1725
2014.0      1697
2012.0      1526
2011.0      1232
2010.0      1199
2009.0      1091
2007.0       886
2008.0       819
2006.0       804
2004.0       797
2005.0       785
2003.0       697
1998.0       675
1997.0       671
1999.0       661
2002.0       623
2001.0       576
1996.0       524
1995.0       423
1994.0       410
2000.0       324
1992.0       322
1993.0       310
1991.0       262
1990.0       218
1989.0       156
1988.0       116
1987.0       104
1986.0        74
1985.0        39
1983.0        38
1984.0        37
1980.0        26
1981.0        23
1982.0        23
2022.0        15
1979.0        12
1974.0         8
1973.0         8
1977.0         4
1976.0         3
1969.0         3
1975.0         3
1971.0         2
1972.0         2
1970.0         1
1968.0         1
1966.0         1
1967.0         1
1965.0         1
1962.0         1
2024.0        

In [17]:
df['stat_iso'].map(value_labels['stat_iso']).value_counts()

CZE    231074
SVK      6271
UKR       862
RUS       602
***       463
        ...  
NGA         1
FIN         1
TUN         1
PHL         1
AGO         1
Name: stat_iso, Length: 88, dtype: int64

In [18]:
df['bydliste_stat_iso'].map(value_labels['bydliste_stat_iso']).value_counts()

CZE    233303
SVK      5913
RUS       270
***       225
UKR       203
        ...  
TUR         1
ETH         1
BGD         1
ATA         1
POL         1
Name: bydliste_stat_iso, Length: 70, dtype: int64

In [119]:
df, variable_labels, value_labels = loader(year=17)
df.to_stata(f'{data_root}/uchazec/uch17.dta', write_index=False, version=118, variable_labels=variable_labels, value_labels=value_labels)

/tmp/ipykernel_401/221791032.py:7: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')


In [56]:
akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
akvo.head()

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AKVO,1101R006,Deskriptivní geometrie,Deskriptivní geometrie,1.1.2003,31.12.2999,None,None
1,AKVO,1101R008,Diskrétní matematika,Diskrétní matematika,1.1.2003,31.12.2999,None,None
2,AKVO,1101R014,Matematická analýza,Matematická analýza,1.1.2003,31.12.2999,11047,None
3,AKVO,1101R016,Matematika,Matematika,1.1.2003,31.12.2999,11617,None
4,AKVO,1101R018,"Matem.,aplikace(analýza)",Matematika a její aplikace (zaměření matematic...,1.1.2003,31.12.2999,None,None


In [66]:
# načti data z roku 2021
df = pd.read_csv(f'{data_root}/{path21}.csv', encoding='cp1250')

/tmp/ipykernel_401/367580044.py:2: DtypeWarning: Columns (3,6,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path21}.csv', encoding='cp1250')


In [67]:
df[:1000].to_csv('temp/uchazec21.csv')

In [79]:
program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
program.to_csv('temp/uchazec/program.csv')

In [80]:
akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
akvo.to_csv('temp/uchazec/akvo.csv')

In [84]:
akko_xml = 'http://stistko.uiv.cz/katalog/textdata/C11240AKKO.xml'
akko = pd.read_xml(akko_xml, encoding='cp1250', xpath='./veta')
akko.to_csv('temp/uchazec/akko2.csv')

In [85]:
regpro = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv', encoding='cp1250')
regpro.to_csv('temp/uchazec/regpro.csv')

In [88]:
pd.to_numeric(df['fak_id'])

ValueError: Unable to parse string "6N900" at position 20

In [109]:
regpro['VS_PLN'] + '___' + regpro['ZAR_PLN']

0      Jihočeská univerzita v Českých Budějovicích   ...
1      Slezská univerzita v Opavě                    ...
2      Česká zemědělská univerzita v Praze           ...
3      Univerzita Palackého v Olomouci               ...
4      Západočeská univerzita v Plzni                ...
                             ...                        
205    Vysoká škola podnikání a práva, a.s.          ...
206    Vysoká škola kreativní komunikace             ...
207    Vysoká škola finanční a správní, a.s.         ...
208    Technická univerzita v Liberci                ...
209    Vysoká škola ekonomie a managementu, a.s.     ...
Length: 210, dtype: object

In [68]:
program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
program.head()

,KOD,NAZEV,RID
0,B0111A190001,Pedagogika ...,14210
1,B0111A190002,Sociální pedagogika a volný čas ...,14410
2,B0111A190003,Andragogika ...,15210
3,B0111A190004,Andragogika a personální řízení ...,11210
4,B0111A190005,Sociální pedagogika a poradenství ...,14210


In [65]:
df['program']

0         M0421A220001
1         B0321A180002
2                M5103
3         M0421A220001
4         B0413P050025
              ...     
284196    B0411P050002
284197    B0111A190023
284198    B0923A240005
284199    B0923A240005
284200           B6739
Name: program, Length: 284201, dtype: object

In [60]:
value_labels['obor1_1']

{1: 'PŘÍRODNÍ VĚDY A NAUKY',
 2: 'TECHNICKÉ VĚDY A NAUKY (1. část)',
 3: 'TECHNICKÉ VĚDY A NAUKY (2. část)',
 4: 'ZEMĚDĚLSKO-LESNICKÉ A VETERINÁRNÍ VĚDY A NAUKY',
 5: 'ZDRAVOTNICTVÍ, LÉKAŘSKÉ A FARMACEUTICKÉ VĚDY A NAUKY',
 6: 'SPOLEČENSKÉ VĚDY, NAUKY A SLUŽBY (1. část)',
 7: 'SPOLEČENSKÉ VĚDY, NAUKY A SLUŽBY (2. část)',
 8: 'VĚDY A NAUKY O KULTUŘE A UMĚNÍ',
 9: 'VOJENSKÉ VĚDY A NAUKY'}

In [62]:
df['obor1_1'].iloc[1]

''

In [42]:
w = pd.io.stata.StataWriterUTF8(f'{data_root}/uchazec/uch21.dta', df, write_index=False, version=118,
                                variable_labels=variable_labels, value_labels=value_labels)
w.write_file()

UnicodeEncodeError: 'latin-1' codec can't encode character '\u0161' in position 11: ordinal not in range(256)

In [43]:
df.to_stata(f'{data_root}/uchazec/uch21.dta', write_index=False, version=118, variable_labels=variable_labels) #, value_labels=value_labels)

In [ ]:
w = pd.io.stata.StataWriterUTF8('foo.dta', df_master)
w.write_file()

In [16]:
df.head()

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,OBOR5,APRO1,APRO2,APRO3,DAT_REG,VYPR,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV
0,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,NaN,NaN,NaN,8072021,15,08072021,1,08072021,1 00 0 0 000000 0000000
1,20211031,1423,,703.0,,83107,703.0,2,999999703.0,,...,,NaN,NaN,NaN,23062021,15,23062021,1,23062021,1 00 0 0 000000 0000000
2,20211031,1411,,703.0,,91624,703.0,2,999999703.0,,...,,NaN,NaN,NaN,28072021,15,28072021,1,28072021,1 00 0 0 000000 0000000
3,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,NaN,NaN,NaN,8072021,15,08072021,1,08072021,1 00 0 0 000000 0000000
4,20211031,78900,,566.0,,,566.0,2,999999566.0,,...,,NaN,NaN,NaN,10112020,20,10112020,0,,00 01 1 0000 0000 0 0


In [44]:
pd.__version__

'1.4.3'

In [17]:
aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')

In [18]:
aast

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AAST,0,Zatím nezjištěno,Zatím nezjištěno,1.1.2000,31.12.2999,***************,pouze pro prezentaci
1,AAST,4,Afghánistán,Afghánská islámská republika,1.7.2014,31.12.2999,AFAFG601 5166020,None
2,AAST,8,Albánie,Albánská republika,5.5.2005,31.12.2999,ALALB111 4107015,None
3,AAST,10,Antarktida,Antarktida,5.5.2005,31.12.2999,AQATA961 5089150,None
4,AAST,12,Alžírsko,Alžírská demokratická a lidová republika,1.1.2011,31.12.2999,DZDZA701 5120830,None
...,...,...,...,...,...,...,...,...
247,AAST,876,Wallis a Futuna,Teritorium Wallisovy ostrovy a Futuna,1.7.2014,31.12.2999,WFWLF9933 5081150,None
248,AAST,882,Samoa,Nezávislý stát Samoa,5.5.2005,31.12.2999,WSWSM931 5081950,None
249,AAST,887,Jemen,Jemenská republika,5.5.2005,31.12.2999,YEYEM623 5165320,None
250,AAST,894,Zambie,Zambijská republika,5.5.2005,31.12.2999,ZMZMB785 5137830,None


In [21]:
aast[['KOD', 'ZKR']].set_index('KOD')['ZKR'].to_dict()

{0: 'Zatím nezjištěno',
 4: 'Afghánistán',
 8: 'Albánie',
 10: 'Antarktida',
 12: 'Alžírsko',
 16: 'Americká Samoa',
 20: 'Andorra',
 24: 'Angola',
 28: 'Antigua a Barbuda',
 31: 'Ázerbájdžán',
 32: 'Argentina',
 36: 'Austrálie',
 40: 'Rakousko',
 44: 'Bahamy',
 48: 'Bahrajn',
 50: 'Bangladéš',
 51: 'Arménie',
 52: 'Barbados',
 56: 'Belgie',
 60: 'Bermudy',
 64: 'Bhútán',
 68: 'Bolívie',
 70: 'Bosna a Hercegovina',
 72: 'Botswana',
 74: 'Bouvetův ostrov',
 76: 'Brazílie',
 84: 'Belize',
 86: 'Britské indickooc.území',
 90: 'Šalomounovy ostrovy',
 92: 'Britské Panenské ostr.',
 95: 'Kosovo',
 96: 'Brunej',
 100: 'Bulharsko',
 104: 'Myanmar',
 108: 'Burundi',
 112: 'Bělorusko',
 116: 'Kambodža',
 120: 'Kamerun',
 124: 'Kanada',
 132: 'Kapverdy',
 136: 'Kajmanské ostrovy',
 140: 'Středoafrická republika',
 144: 'Šrí Lanka',
 148: 'Čad',
 152: 'Chile',
 156: 'Čína',
 158: 'Tchaj-wan',
 162: 'Vánoční ostrov',
 166: 'Kokosové (Keeling.) ostr.',
 170: 'Kolumbie',
 174: 'Komory',
 175: 'Mayott

In [7]:
to_rename = {
    'RDAT': 'dat_sber',
    'RID': 'fak_id',
    'ROD_KOD': 'id',
    'STATB': 'bydliste_stat',
    'OBECB': 'bydliste_obec',
    'PSCB': 'bydliste_psc',
    'STAT': 'stat',
    'ODHL': 'odhl',
    'IZOS': 'ss_izo',
    'OBSS': 'ss_obor',
    'RMAT': 'rmat',
    'TYP_ST': 'typ_st',
    'FORMA_ST': 'forma_st',
    'PROGRAM': 'program',
    'OBOR1': 'obor1',
    'OBOR2': 'obor2',
    'OBOR3': 'obor3',
    'OBOR4': 'obor4',
    'OBOR5': 'obor5',
    'APRO1': 'apro1',
    'APRO2': 'apro2',
    'APRO3': 'apro3',
    'DAT_REG': 'dat_reg',
    'VYPR': 'vypr',
    'DAT_VYPR': 'dat_vypr',
    'ZAPS': 'zaps',
    'DAT_ZAPS': 'dat_zaps'
}

to_drop = ['CHYV']

In [12]:
# conversion to numeric
for c in ['RMAT', 'STAT', 'STATB', 'IZOS'] + [f'APRO{i}' for i in range(1, 4)]:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# strip white spaces
for c in ['OBSS', 'RID', 'PROGRAM'] + [f'OBOR{i}' for i in range(1, 6)]:
    df[c] = df[c].str.strip()

df = df.rename(columns=to_rename).drop(columns=to_drop)

In [13]:
df.head()

,dat_sber,fak_id,id,bydliste_stat,bydliste_obec,bydliste_psc,stat,odhl,ss_izo,ss_obor,...,obor4,obor5,apro1,apro2,apro3,dat_reg,vypr,dat_vypr,zaps,dat_zaps
0,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,8072021,15,08072021,1,08072021
1,20211031,1423,,703.0,,83107,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,23062021,15,23062021,1,23062021
2,20211031,1411,,703.0,,91624,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,28072021,15,28072021,1,28072021
3,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,8072021,15,08072021,1,08072021
4,20211031,78900,,566.0,,,566.0,2,999999566.0,,...,,,NaN,NaN,NaN,10112020,20,10112020,0,


## Loader

In [303]:
def loader(year=17):
    # year can be 17 or 21
    data_root = '/mnt/d/projects/idea/data'
    path = f'uchazec/0022MUCH{year}P'
    
    df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')

    # conversion to numeric
    for c in ['RMAT', 'STAT', 'STATB', 'IZOS'] + [f'APRO{i}' for i in range(1, 4)]:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        
    # strip white spaces
    for c in ['OBSS', 'RID', 'PROGRAM'] + [f'OBOR{i}' for i in range(1, 6)]:
        df[c] = df[c].str.strip()
    
    # label STAT, STATB: register AAST
    aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
    aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')
    aast['ISO_ALPHA3'] = aast['SPC'].str[2:5]
    df = pd.merge(df, aast[['KOD', 'ZKR', 'ISO_ALPHA3']].rename(columns={'KOD': 'STAT', 'ZKR': 'STAT_LABEL', 'ISO_ALPHA3': 'STAT_ISO'}), 
              how='left')
    df = pd.merge(df, aast[['KOD', 'ZKR', 'ISO_ALPHA3']].rename(columns={'KOD': 'STATB', 'ZKR': 'STATB_LABEL', 'ISO_ALPHA3': 'STATB_ISO'}), 
              how='left')
    
    # label OBSS - I am not including full school info
    #   -> actually, this give me slightly better match - probably because I am including also specializations that are not valid anymore
    li = []
    for y in range(1999, 2023):
        pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
        li.append(pdf)
    ss = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)
    
    def selector(frame):
        # just use the longest one - the shorter ones typically use abbreviations
        frame['len'] = frame['PLN_NAZ'].str.strip().str.len()
        return frame.sort_values('len', ascending=False)['PLN_NAZ'].iloc[0]

    ss['OBOR'] = ss['OBOR'].str.strip()
    obory = ss[['OBOR', 'PLN_NAZ']].drop_duplicates()
    obory_uni = obory.groupby('OBOR')[['PLN_NAZ']].apply(selector).reset_index()
    df = pd.merge(df, obory_uni.rename(columns={'OBOR': 'OBSS', 0: 'OBSS_LABEL'}), how='left')
    
    sss = ss[['IZO', 'ZAR_PLN', 'VUSC']].drop_duplicates()
    df = pd.merge(df, sss.rename(columns={'IZO': 'IZOS', 'ZAR_PLN': 'IZOS_LABEL', 'VUSC': 'IZOS_NUTS'}), how='left')
    
    registers = {
        'ODHL': 'http://stistko.uiv.cz/katalog/textdata/C213729MCPP.xml',
        'TYP_ST': 'http://stistko.uiv.cz/katalog/textdata/C214019PASP.xml',
        'FORMA_ST' : 'http://stistko.uiv.cz/katalog/textdata/C214048PAFS.xml',
        'VYPR': 'http://stistko.uiv.cz/katalog/textdata/C214147MCPR.xml',
        'ZAPS': 'http://stistko.uiv.cz/katalog/textdata/C21427MCZR.xml',
        # 'OBSS': 'http://stistko.uiv.cz/katalog/textdata/C113922AKSO.xml'
    }
    
    for c, url in registers.items():
        rg = pd.read_xml(url, encoding='cp1250', xpath='./veta')
        df = pd.merge(df, rg[['KOD', 'TXT']].rename(columns={'KOD': c, 'TXT': f'{c}_LABEL'}), how='left')

    regpro = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv', encoding='cp1250')
    regpro['RID'] = regpro['RID'].str.strip()
    regpro_cols = ['ZAR_PLN', 'VS_PLN', 'ZAR_NAZ', 'UZEMI']
    regpro = regpro[['RID'] + regpro_cols].rename(columns={x: f'RID_{x}' for x in regpro_cols})
    df = pd.merge(df, regpro.rename(columns={'RID_UZEMI': 'RID_NUTS'}), how='left')
    
    program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
    df = pd.merge(df, program.rename(columns={'KOD': 'PROGRAM', 'NAZEV': 'PROGRAM_LABEL'}), how='left')
    
    # plné obory
    akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
    akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
    for i in range(1, 6):
        df = pd.merge(df, akvo[['KOD', 'TXT']].rename(columns={'KOD': f'OBOR{i}', 'TXT': f'OBOR{i}_LABEL'}), how='left')
        
    # aprobace
    aaap_xml = 'http://stistko.uiv.cz/katalog/textdata/C214234AAAP.xml'
    aaap = pd.read_xml(aaap_xml, encoding='cp1250', xpath='./veta')
    for i in range(1, 4):
        df = pd.merge(df, aaap[['KOD', 'TXT']].rename(columns={'KOD': f'APRO{i}', 'TXT': f'APRO{i}_LABEL'}), how='left')
        
    # studijní obory podle skupin
    akko_xml = 'http://stistko.uiv.cz/katalog/textdata/C11240AKKO.xml'
    akko = pd.read_xml(akko_xml, encoding='cp1250', xpath='./veta')
    for i in [1, 2, 5]:
        df[f'OBOR1_{i}'] = df['OBOR1'].str[:i]
        df = pd.merge(df, akko[['KOD', 'TXT']].rename(columns={'KOD': f'OBOR1_{i}', 'TXT': f'OBOR1_{i}_LABEL'}), how='left')
    
    df['TYP_SS'] = np.where(df['OBSS'] == '', 'JINE', np.where(df['OBSS'].str.startswith('794'), 'G', 'SOS'))
    df['TYP_G'] = np.where(df['TYP_SS'] != 'G', np.nan, df['OBSS'].str[5])
    
    # fillna in some columns
    for c in ['PROGRAM_LABEL'] + [f'OBOR{i}_LABEL' for i in range(1, 6)] + [f'APRO{i}_LABEL' for i in range(1, 4)]:
        df[c] = df[c].fillna('')
    
    return df